In [1]:
#pip install pandas

In [5]:
############################################################
# 🧠 Data Extraction + OpenAI Analysis Pipeline (Python)
# Author: Shawn
# Description:
#   Converts R ODBC / foreach / doParallel workflow
#   into a Python + OpenAI integration pipeline.
############################################################

import os
import pyodbc
import pandas as pd
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
import time

# ==========================================================
# ✅ 1️⃣ Environment & API Setup (Hardcoded Key)
# ==========================================================
# ⚠️ Note: Hardcoding API keys is only recommended for local testing.
# Store it in environment variables or .env for production.
API_KEY = "*"

os.environ["OPENAI_API_KEY"] = API_KEY
client = OpenAI(api_key=API_KEY)

# ==========================================================
# ✅ 2️⃣ Detect Available Cores
# ==========================================================
num_cores = multiprocessing.cpu_count() - 1
print(f"💻 Using {num_cores} cores for data processing")

# ==========================================================
# ✅ 3️⃣ SQL Query Definition
# ==========================================================
qry = """
SELECT DISTINCT
CUST.MAILTYPE,
CUST.IID,
CUST.IID_TXT,

CASE WHEN CUST.QUOTE = 1 AND CUST.DIR_IND = 1 THEN 1 ELSE 0 END AS QUOTE,
CUST.QUOTE AS DIR_OR_IND_QT,
CUST.DIR_IND AS DIR_QT_FLG,
CUST.BOUND,

--CUST.TM,
CUST.TQ, 
--CUST.TM + CUST.TQ AS TMTQ,

CUST.CAMP_NBR,
CUST.CAMPAIGN_NUMBER,
-- Weight = number of weeks since campaign
(CAST(LEFT(CUST.CAMP_NBR, 2) AS INT) + 2000) * 52 + CAST(RIGHT(CUST.CAMP_NBR, 2) AS INT) 
 - (YEAR(GETDATE()) * 52 + DATEPART(WEEK, GETDATE())) AS CAMP_WEEK_WEIGHT,

CUST.KEYCODE,

CUST.TARGET_MAIL_WK,
SUBSTRING(CAST(CUST.TARGET_MAIL_WK AS VARCHAR), 6, 2) AS MAIL_MONTH,
FORMAT(CUST.TARGET_MAIL_WK, 'yyyy-MM') AS MAIL_YM,

--CUST.FIRST_NAME,
--CUST.LAST_NAME,
--CUST.ADDRESS_LINE_1,
--ISNULL(CUST.ADDRESS_LINE_2, '') AS ADDRESS_LINE_2,
--CUST.CITY,
--CUST.STATE,
--CUST.ZIP_5,

CASE 
WHEN UPPER(CUST.ADDRESS_LINE1) LIKE '%APT%' THEN 0
WHEN UPPER(ISNULL(CUST.ADDRESS_LINE2, '')) LIKE '%APT%' THEN 0
ELSE 1 END AS HOMEOWNER,

ISNULL(CUST.MARRIED, 1) * 
ISNULL(CASE WHEN UPPER(CUST.ADDRESS_LINE1) LIKE '%APT%' THEN 0 WHEN UPPER(ISNULL(CUST.ADDRESS_LINE2, '')) LIKE '%APT%' THEN 0 ELSE 1 END, 1)
AS MARRIED_HOMEOWNER,

ISNULL(CASE WHEN CUST.RA_PARTNER_ACTIVE = 'Y' THEN 1 WHEN CUST.RA_PARTNER_ACTIVE = 'N' THEN 0 ELSE 0 END, 1) *
ISNULL(CUST.TQ, 1) AS RA_ACTIVE_TQ,

CUST.STATE_INDEX_NBR,

CUST.GS_EXPIRE_DATE,
CUST.GS_LAST_ACTIVITY_DATE,

ISNULL(
  DATEDIFF(
    DAY,
    TRY_CAST(CUST.TARGET_MAIL_WK AS DATE),
    CUST.GS_LAST_ACTIVITY_DATE
  ),
  0
) AS GS_LAST_ACTIVITY_DATE_DIFF,

ISNULL(
  DATEDIFF(
    DAY,
    TRY_CAST(CUST.TARGET_MAIL_WK AS DATE),
    CUST.GS_EXPIRE_DATE
  ),
  0
) AS GS_EXPIRE_DATE_DIFF,

CASE 
WHEN ISNULL(CUST.AGE, CUST.AGE_2) = '' THEN '59.5'
WHEN ISNULL(CUST.AGE, CUST.AGE_2) IS NULL THEN '59.5'
ELSE ISNULL(CUST.AGE, CUST.AGE_2) 
END AS AGE,

CASE WHEN CUST.ALL_CWH_0_12_AVG_SPEND = '' THEN 0.0 WHEN CUST.ALL_CWH_0_12_AVG_SPEND IS NULL THEN 0.0 

ELSE CAST(CUST.ALL_CWH_0_12_AVG_SPEND AS FLOAT) END AS ALL_CWH_0_12_AVG_SPEND,

--CUST.ADDRESS_LINE1 + ' ' + ISNULL(CUST.ADDRESS_LINE2, '') + ' ' + CUST.ZIP_5 AS ADDRS, 

CUST.POSTAL_CODE_5, SUBSTRING(CUST.POSTAL_CODE_5, 0, 3) AS ZIP_2, SUBSTRING(CUST.POSTAL_CODE_5, 0, 4) AS ZIP_3, CUST.DIV_NBR,

CASE WHEN CUST.RV_MAG_ACTIVE = 'Y' THEN 1 WHEN CUST.RV_MAG_ACTIVE = 'N' THEN 0 ELSE 0 END AS RV_MAG_ACTIVE,
CASE WHEN CUST.RV_SERVICE_0_12M = 'Y' THEN 1 WHEN CUST.RV_SERVICE_0_12M = 'N' THEN 0 ELSE 0 END AS RV_SERVICE_0_12M,

CASE WHEN CUST.TOTAL_CLAIMS = '' THEN 0 WHEN CUST.TOTAL_CLAIMS IS NULL THEN 0 ELSE CUST.TOTAL_CLAIMS END AS TOTAL_CLAIMS,

ISNULL(CUST.GS_MEMBERSHIP_STATUS, 0) AS GS_MEMBERSHIP_STATUS,
ISNULL(CUST.GS_MEMBER_TYPE, 0) AS GS_MEMBER_TYPE,
ISNULL(CUST.GS_ORIG_START_DATE, 0) AS GS_ORIG_START_DATE, 
ISNULL(CUST.GS_CURRENT_START_DATE, 0) AS GS_CURRENT_START_DATE, 
ISNULL(CUST.GS_CURRENT_TERM, 0) AS GS_CURRENT_TERM,
ISNULL(CUST.GS_PRIOR_TERM, 0) AS GS_PRIOR_TERM, 
ISNULL(CUST.GS_CLUB_TIMES_RENEWED, 0) AS GS_CLUB_TIMES_RENEWED,

CASE WHEN CUST.GS_ERS_MEMBERSHIP_STATUS IS NULL THEN 0 ELSE CUST.GS_ERS_MEMBERSHIP_STATUS END AS GS_ERS_MEMBERSHIP_STATUS,
CASE WHEN CUST.GS_ERS_MEMBER_TYPE IS NULL THEN 0 ELSE CUST.GS_ERS_MEMBER_TYPE END AS GS_ERS_MEMBER_TYPE,
CASE WHEN CUST.GS_ERS_ORIG_START_DATE IS NULL THEN 0 ELSE CUST.GS_ERS_ORIG_START_DATE END AS GS_ERS_ORIG_START_DATE,
CASE WHEN CUST.GS_ERS_CURRENT_START_DATE IS NULL THEN 0 ELSE CUST.GS_ERS_CURRENT_START_DATE END AS GS_ERS_CURRENT_START_DATE,
CASE WHEN CUST.GS_ERS_CURRENT_TERM IS NULL THEN 0 ELSE CUST.GS_ERS_CURRENT_TERM END AS GS_ERS_CURRENT_TERM,
CASE WHEN CUST.GS_ERS_PRIOR_TERM IS NULL THEN 0 ELSE CUST.GS_ERS_PRIOR_TERM END AS GS_ERS_PRIOR_TERM,
CASE WHEN CUST.GS_ERS_TIMES_RENEWED IS NULL THEN 0 ELSE CUST.GS_ERS_TIMES_RENEWED END AS GS_ERS_TIMES_RENEWED,
CASE WHEN CUST.GS_ERS_LAST_ACTIVITY_DT IS NULL THEN 0 ELSE CUST.GS_ERS_LAST_ACTIVITY_DT END AS GS_ERS_LAST_ACTIVITY_DT,
CASE WHEN CUST.CSP_MEMBERSHIP_STATUS IS NULL THEN 0 ELSE CUST.CSP_MEMBERSHIP_STATUS END AS CSP_MEMBERSHIP_STATUS,
CASE WHEN CUST.CSP_MEMBER_TYPE IS NULL THEN 0 ELSE CUST.CSP_MEMBER_TYPE END AS CSP_MEMBER_TYPE,
CASE WHEN CUST.CSP_CURRENT_START_DATE IS NULL THEN 0 ELSE CUST.CSP_CURRENT_START_DATE END AS CSP_CURRENT_START_DATE,
CASE WHEN CUST.CSP_CURRENT_TERM IS NULL THEN 0 ELSE CUST.CSP_CURRENT_TERM END AS CSP_CURRENT_TERM,
CASE WHEN CUST.CSP_PRIOR_TERM IS NULL THEN 0 ELSE CUST.CSP_PRIOR_TERM END AS CSP_PRIOR_TERM,
CASE WHEN CUST.CSP_TIMES_RENEWED IS NULL THEN 0 ELSE CUST.CSP_TIMES_RENEWED END AS CSP_TIMES_RENEWED,
CASE WHEN CUST.CSP_LAST_ACTIVITY_DATE IS NULL THEN 0 ELSE CUST.CSP_LAST_ACTIVITY_DATE END AS CSP_LAST_ACTIVITY_DATE,
CASE WHEN CUST.RV_ENTHUSIAST_FLAG = 'Y' THEN 1 WHEN CUST.RV_ENTHUSIAST_FLAG = 'N' THEN 0 ELSE 0 END AS RV_ENTHUSIAST_FLAG,
CASE WHEN CUST.FISHING_MARINE_FLAG = 'Y' THEN 1 WHEN CUST.FISHING_MARINE_FLAG = 'N' THEN 0 ELSE 0 END AS FISHING_MARINE_FLAG,
CASE WHEN CUST.HUNT_SHOOT_FLAG = 'Y' THEN 1 WHEN CUST.HUNT_SHOOT_FLAG = 'N' THEN 0 ELSE 0 END AS HUNT_SHOOT_FLAG,
CASE WHEN CUST.OUTDOOR_FLAG = 'Y' THEN 1 WHEN CUST.OUTDOOR_FLAG = 'N' THEN 0 ELSE 0 END AS OUTDOOR_FLAG,
CASE WHEN CUST.CW_WARRANTY_PRODUCT IS NULL THEN 0 ELSE CUST.CW_WARRANTY_PRODUCT END AS CW_WARRANTY_PRODUCT,
ISNULL(CUST.WOODALL_SUBS_FLAG, 0) AS WOODALL_SUBS_FLAG, 
CASE WHEN CUST.CREDIT_CARD_HOLDER_DIVISION IS NULL THEN 0.0 ELSE CAST(CUST.CREDIT_CARD_HOLDER_DIVISION AS FLOAT) END AS CREDIT_CARD_HOLDER_DIVISION,
CASE WHEN CUST.ALL_CWH_0_12_AMT_PURCH_TOTAL IS NULL THEN 0.0 ELSE CAST(CUST.ALL_CWH_0_12_AMT_PURCH_TOTAL AS FLOAT) END AS ALL_CWH_0_12_AMT_PURCH_TOTAL,
CASE WHEN CUST.CW_0_12_AVG_SPEND IS NULL THEN 0.0 ELSE CAST(CUST.CW_0_12_AVG_SPEND AS FLOAT) END AS CW_0_12_AVG_SPEND,
CASE WHEN CUST.CW_TRANS_12_MONTHS IS NULL THEN 0.0 ELSE CAST(CUST.CW_TRANS_12_MONTHS AS FLOAT) END AS CW_TRANS_12_MONTHS,
CASE WHEN CUST.CW_0_12_AMT_PURCH_TOTAL IS NULL THEN 0.0 ELSE CAST(CUST.CW_0_12_AMT_PURCH_TOTAL AS FLOAT) END AS CW_0_12_AMT_PURCH_TOTAL,
CASE WHEN CUST.GANDER_0_12_AVG_SPEND IS NULL THEN 0.0 ELSE CAST(CUST.GANDER_0_12_AVG_SPEND AS FLOAT) END AS GANDER_0_12_AVG_SPEND,
CASE WHEN CUST.GANDER_TRANS_12_MONTHS IS NULL THEN 0.0 ELSE CAST(CUST.GANDER_TRANS_12_MONTHS AS FLOAT) END AS GANDER_TRANS_12_MONTHS,
CASE WHEN CUST.GANDER_0_12_AMT_PURCH_TOTAL IS NULL THEN 0.0 ELSE CAST(CUST.GANDER_0_12_AMT_PURCH_TOTAL AS FLOAT) END AS GANDER_0_12_AMT_PURCH_TOTAL,
CASE WHEN CUST.OVERTONS_TRANS_12_MONTHS IS NULL THEN 0.0 ELSE CAST(CUST.OVERTONS_TRANS_12_MONTHS AS FLOAT) END AS OVERTONS_TRANS_12_MONTHS,
CASE WHEN CUST.OVERTONS_0_12_AMT_PURCH_TOTAL IS NULL THEN 0.0 ELSE CAST(CUST.OVERTONS_0_12_AMT_PURCH_TOTAL AS FLOAT) END AS OVERTONS_0_12_AMT_PURCH_TOTAL,
CASE WHEN CUST.ALL_CWH_0_12_PURE_RTL_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_0_12_PURE_RTL_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_0_12_PURE_RTL_BUYERS,
CASE WHEN CUST.ALL_CWH_13_24_PURE_RTL_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_13_24_PURE_RTL_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_13_24_PURE_RTL_BUYERS,
CASE WHEN CUST.ALL_CWH_0_12_INT_MO_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_0_12_INT_MO_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_0_12_INT_MO_BUYERS,
CASE WHEN CUST.ALL_CWH_13_24_INT_MO_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_13_24_INT_MO_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_13_24_INT_MO_BUYERS,
CASE WHEN CUST.ALL_CWH_0_12_COMBO_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_0_12_COMBO_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_0_12_COMBO_BUYERS,
CASE WHEN CUST.ALL_CWH_13_24_COMBO_BUYERS = 'Y' THEN 1 WHEN CUST.ALL_CWH_13_24_COMBO_BUYERS = 'N' THEN 0 ELSE 0 END AS ALL_CWH_13_24_COMBO_BUYERS,
CASE WHEN CUST.CATALOG_REQ_0_24M = 'Y' THEN 1 WHEN CUST.CATALOG_REQ_0_24M = 'N' THEN 0 ELSE 0 END AS CATALOG_REQ_0_24M,
CASE WHEN CUST.EVENT_ACTIVITY_0_12M = 'Y' THEN 1 WHEN CUST.EVENT_ACTIVITY_0_12M = 'N' THEN 0 ELSE 0 END AS EVENT_ACTIVITY_0_12M,
CASE WHEN CUST.RV_SALES_0_12M = 'Y' THEN 1 WHEN CUST.RV_SALES_0_12M = 'N' THEN 0 ELSE 0 END AS RV_SALES_0_12M,
CASE WHEN CUST.RV_SALES_13_24M = 'Y' THEN 1 WHEN CUST.RV_SALES_13_24M = 'N' THEN 0 ELSE 0 END AS RV_SALES_13_24M,
CASE WHEN CUST.RV_SERVICE_13_24M = 'Y' THEN 1 WHEN CUST.RV_SERVICE_13_24M = 'N' THEN 0 ELSE 0 END AS RV_SERVICE_13_24M,
CASE WHEN CUST.GSC_ACTIVE = 'Y' THEN 1 WHEN CUST.GSC_ACTIVE = 'N' THEN 0 ELSE 0 END AS GSC_ACTIVE,
CASE WHEN CUST.GSC_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.GSC_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS GSC_24M_EXPIRES,
CASE WHEN CUST.CTC_ACTIVE = 'Y' THEN 1 WHEN CUST.CTC_ACTIVE = 'N' THEN 0 ELSE 0 END AS CTC_ACTIVE,
CASE WHEN CUST.CTC_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.CTC_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS CTC_24M_EXPIRES,
CASE WHEN CUST.RA_DLR_ACTIVE = 'Y' THEN 1 WHEN CUST.RA_DLR_ACTIVE = 'N' THEN 0 ELSE 0 END AS RA_DLR_ACTIVE,
CASE WHEN CUST.RA_DLR_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.RA_DLR_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS RA_DLR_24M_EXPIRES,
CASE WHEN CUST.GSRA_AUTO_ACTIVE = 'Y' THEN 1 WHEN CUST.GSRA_AUTO_ACTIVE = 'N' THEN 0 ELSE 0 END AS GSRA_AUTO_ACTIVE,
CASE WHEN CUST.GSRA_RV_ACTIVE = 'Y' THEN 1 WHEN CUST.GSRA_RV_ACTIVE = 'N' THEN 0 ELSE 0 END AS GSRA_RV_ACTIVE,
CASE WHEN CUST.GSRA_RV_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.GSRA_RV_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS GSRA_RV_24M_EXPIRES,
CASE WHEN CUST.RA_MANUF_ACTIVE = 'Y' THEN 1 WHEN CUST.RA_MANUF_ACTIVE = 'N' THEN 0 ELSE 0 END AS RA_MANUF_ACTIVE,
CASE WHEN CUST.RA_MANUF_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.RA_MANUF_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS RA_MANUF_24M_EXPIRES,
CASE WHEN CUST.RA_PARTNER_ACTIVE = 'Y' THEN 1 WHEN CUST.RA_PARTNER_ACTIVE = 'N' THEN 0 ELSE 0 END AS RA_PARTNER_ACTIVE,
CASE WHEN CUST.ESP_ACTIVE = 'Y' THEN 1 WHEN CUST.ESP_ACTIVE = 'N' THEN 0 ELSE 0 END AS ESP_ACTIVE,
CASE WHEN CUST.ESP_24M_LAPSED = 'Y' THEN 1 WHEN CUST.ESP_24M_LAPSED = 'N' THEN 0 ELSE 0 END AS ESP_24M_LAPSED,
CASE WHEN CUST.ASU_ESP_ACTIVE_POLICIES = 'Y' THEN 1 WHEN CUST.ASU_ESP_ACTIVE_POLICIES = 'N' THEN 0 ELSE 0 END AS ASU_ESP_ACTIVE_POLICIES,
CASE WHEN CUST.ASU_ESP_24M_LAPSED_POLICIES = 'Y' THEN 1 WHEN CUST.ASU_ESP_24M_LAPSED_POLICIES = 'N' THEN 0 ELSE 0 END AS ASU_ESP_24M_LAPSED_POLICIES,
CASE WHEN CUST.GSTA_ACTIVE = 'Y' THEN 1 WHEN CUST.GSTA_ACTIVE = 'N' THEN 0 ELSE 0 END AS GSTA_ACTIVE,
CASE WHEN CUST.GSTA_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.GSTA_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS GSTA_24M_EXPIRES,
CASE WHEN CUST.GSTA_DLR_ACTIVE = 'Y' THEN 1 WHEN CUST.GSTA_DLR_ACTIVE = 'N' THEN 0 ELSE 0 END AS GSTA_DLR_ACTIVE,
CASE WHEN CUST.GSTA_DLR_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.GSTA_DLR_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS GSTA_DLR_24M_EXPIRES,
CASE WHEN CUST.GSRA_AUTO_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.GSRA_AUTO_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS GSRA_AUTO_24M_EXPIRES,
CASE WHEN CUST.AGIA_24M_LAPSED_POLICIES = 'Y' THEN 1 WHEN CUST.AGIA_24M_LAPSED_POLICIES = 'N' THEN 0 ELSE 0 END AS AGIA_24M_LAPSED_POLICIES,
CASE WHEN CUST.RV_MAG_24M_EXPIRES = 'Y' THEN 1 WHEN CUST.RV_MAG_24M_EXPIRES = 'N' THEN 0 ELSE 0 END AS RV_MAG_24M_EXPIRES,
CASE WHEN CUST.CC_ALL_FLAVORS_OPEN_ACCTS = 'Y' THEN 1 WHEN CUST.CC_ALL_FLAVORS_OPEN_ACCTS = 'N' THEN 0 ELSE 0 END AS CC_ALL_FLAVORS_OPEN_ACCTS,
CASE WHEN CUST.PPP_ACTIVE_POLICIES = 'Y' THEN 1 WHEN CUST.PPP_ACTIVE_POLICIES = 'N' THEN 0 ELSE 0 END AS PPP_ACTIVE_POLICIES,
CASE WHEN CUST.PPP_24M_LAPSED_POLICIES = 'Y' THEN 1 WHEN CUST.PPP_24M_LAPSED_POLICIES	 = 'N' THEN 0 ELSE 0 END AS PPP_24M_LAPSED_POLICIES,

CASE WHEN CUST.MEDIAN_INCOME = '' THEN 0 WHEN CUST.MEDIAN_INCOME = 'NA' THEN 0 ELSE ISNULL(CUST.MEDIAN_INCOME, 0) END AS MEDIAN_INCOME,
CASE WHEN CUST.DENSITY = '' THEN 0 WHEN CUST.DENSITY = 'NA' THEN 0 ELSE ISNULL(CUST.DENSITY, 0) END AS DENSITY,
CASE WHEN CUST.MED_HOME_VALUE IS NULL THEN 0 WHEN CUST.MED_HOME_VALUE = 'NA' THEN 0 ELSE ISNULL(CUST.MED_HOME_VALUE, 0) END AS MED_HOME_VALUE,

CUST.MARRIED, ISNULL(CUST.RUCA1, 0) AS RUCA, CUST.RV_TYPE_NBR, 

CUST.NRI_RR, ISNULL(CUST.LAT, 0) AS LAT, ISNULL(CUST.LNG, 0) AS LNG, ISNULL(CUST.LAT, 0) + ISNULL(CUST.LNG, 0) AS GEO_ID, CUST.MEAN_ST_TEMP,

CAST(ISNULL(DATEDIFF(DAY, CUST.GS_ORIG_START_DATE, CAST(GETDATE() AS DATE)), 0) AS INT) AS MEMBERSHIP_TENURE
  
FROM (

SELECT ISNULL(CUST.QUOTE, 0) AS QUOTE, CUST.MAILTYPE,

CASE 
WHEN CUST.CAMP_PCMC = CUST.QT_PCMC AND (CUST.CAMP_PCMC + CUST.QT_PCMC ) IS NOT NULL THEN '1' 
WHEN CUST.CAMP_PCMC <> CUST.QT_PCMC AND (CUST.CAMP_PCMC + CUST.QT_PCMC ) IS NOT NULL THEN '0'
ELSE 0 END AS DIR_IND,

ISNULL(CUST.POLICYBOUGHT, 0) AS BOUND,

CUST.TM, CUST.TQ,

CUST.CAMPAIGN_NUMBER, CUST.CAMP_NBR, CUST.DROP_DATE AS TARGET_MAIL_WK, 
CUST.IID AS IID, '~' + CUST.IID AS IID_TXT, RIGHT(CUST.KC, 3) AS KEYCODE, CUST.IB_AGE AS AGE, CUST.AGE AS AGE_2,
CUST.MEM_NBR, CUST.GS_LAST_ACTIVITY_DATE, CUST.GS_EXPIRE_DATE, CUST.GS_MEMBERSHIP_STATUS, CUST.GS_MEMBER_TYPE,
CUST.GS_ORIG_START_DATE, CUST.GS_CURRENT_START_DATE, CUST.GS_CURRENT_TERM, CUST.GS_PRIOR_TERM, CUST.GS_CLUB_TIMES_RENEWED, CUST.GS_ERS_MEMBERSHIP_STATUS,
CUST.GS_ERS_MEMBER_TYPE, CUST.GS_ERS_ORIG_START_DATE, CUST.GS_ERS_CURRENT_START_DATE, CUST.GS_ERS_CURRENT_TERM, CUST.GS_ERS_PRIOR_TERM, 
CUST.GS_ERS_TIMES_RENEWED, CUST.GS_ERS_LAST_ACTIVITY_DT, CUST.CSP_MEMBERSHIP_STATUS, CUST.CSP_MEMBER_TYPE, CUST.CSP_CURRENT_START_DATE, 
CUST.CSP_CURRENT_TERM, CUST.CSP_PRIOR_TERM, CUST.CSP_TIMES_RENEWED, CUST.CSP_LAST_ACTIVITY_DATE, CUST.RV_ENTHUSIAST_FLAG, CUST.FISHING_MARINE_FLAG,
CUST.HUNT_SHOOT_FLAG, CUST.OUTDOOR_FLAG, CUST.CW_WARRANTY_PRODUCT, CUST.WOODALL_SUBS_FLAG, CUST.TOTAL_CLAIMS, CUST.CREDIT_CARD_HOLDER_DIVISION,
CUST.ALL_CWH_0_12_AVG_SPEND, CUST.ALL_CWH_0_12_AMT_PURCH_TOTAL, CUST.CW_0_12_AVG_SPEND,	CUST.CW_TRANS_12_MONTHS, 
CUST.CW_0_12_AMT_PURCH_TOTAL, CUST.GANDER_0_12_AVG_SPEND, CUST.GANDER_TRANS_12_MONTHS, CUST.GANDER_0_12_AMT_PURCH_TOTAL,
CUST.OVERTONS_TRANS_12_MONTHS, CUST.OVERTONS_0_12_AMT_PURCH_TOTAL, CUST.ALL_CWH_0_12_PURE_RTL_BUYERS, CUST.ALL_CWH_13_24_PURE_RTL_BUYERS,
CUST.ALL_CWH_0_12_INT_MO_BUYERS, CUST.ALL_CWH_13_24_INT_MO_BUYERS, CUST.ALL_CWH_0_12_COMBO_BUYERS, CUST.ALL_CWH_13_24_COMBO_BUYERS, 
CUST.CATALOG_REQ_0_24M, CUST.EVENT_ACTIVITY_0_12M, CUST.RV_SALES_0_12M, CUST.RV_SALES_13_24M, CUST.RV_SERVICE_0_12M,
CUST.RV_SERVICE_13_24M, CUST.GSC_ACTIVE, CUST.GSC_24M_EXPIRES, CUST.CTC_ACTIVE, CUST.CTC_24M_EXPIRES, CUST.RA_DLR_ACTIVE,
CUST.RA_DLR_24M_EXPIRES, CUST.GSRA_AUTO_ACTIVE, CUST.GSRA_RV_ACTIVE, CUST.GSRA_RV_24M_EXPIRES, CUST.RA_MANUF_ACTIVE,
CUST.RA_MANUF_24M_EXPIRES, CUST.RA_PARTNER_ACTIVE, CUST.ESP_ACTIVE, CUST.ESP_24M_LAPSED, CUST.ASU_ESP_ACTIVE_POLICIES,
CUST.ASU_ESP_24M_LAPSED_POLICIES, CUST.GSTA_ACTIVE, CUST.GSTA_24M_EXPIRES, CUST.GSTA_DLR_ACTIVE, CUST.GSTA_DLR_24M_EXPIRES,
CUST.GSRA_AUTO_24M_EXPIRES, CUST.AGIA_24M_LAPSED_POLICIES, CUST.RV_MAG_ACTIVE, CUST.RV_MAG_24M_EXPIRES, CUST.CC_ALL_FLAVORS_OPEN_ACCTS,
CUST.PPP_ACTIVE_POLICIES, CUST.PPP_24M_LAPSED_POLICIES, CUST.ADDRESS_LINE1 AS ADDRESS_LINE1, CUST.ADDRESS_LINE2 AS ADDRESS_LINE2, 
ISNULL(CUST.POSTAL_CODE_5, CUST.ZIP_5) AS POSTAL_CODE_5, 

CUST.RV_TYPE, 
CASE WHEN CUST.RV_TYPE = '' THEN '0' 
WHEN CUST.RV_TYPE = 'A' THEN '10' WHEN CUST.RV_TYPE = 'B' THEN '11' 
WHEN CUST.RV_TYPE = 'C' THEN '13' WHEN CUST.RV_TYPE = 'D' THEN '14' ELSE ISNULL(CUST.RV_TYPE, 0) END AS RV_TYPE_NBR,

CASE WHEN CUST.MARITALSTATUS IN ('1M', '5S', '5M', '5U', '0U') THEN '1' ELSE '0' END AS MARRIED,

CUST.MED_HOME_VALUE, CUST.MEDIAN_INCOME, CUST.DENSITY, CUST.LAT, CUST.LNG, CUST.RISK_RATNG, CUST.NRI_RR, 

CUST.DIV_NBR, CUST.STATE_INDEX_NBR, CUST.RUCA1, CUST.MEAN_ST_TEMP

FROM (
SELECT DISTINCT CAMP.OMNIVIEW_IND_ID AS IID, CAMP.KEYCODE AS KC, CAMP.ZIP_5, 
ROW_NUMBER() OVER (PARTITION BY CAMP.CAMPAIGN_NUMBER, CAMP.OMNIVIEW_IND_ID ORDER BY CAMP.CAMPAIGN_NUMBER DESC) AS R,

Q.QUOTEMADE AS QUOTE, Q.POLICYBOUGHT, TM.TM, TM.TQ,
Q.CAMPAIGNPC + Q.CAMPAIGNMC AS CAMP_PCMC, Q.QPPC + Q.QPMC AS QT_PCMC,
(CAMP.MAILTYPE) AS MAILTYPE, CAST(CAMP.DROP_DATE AS DATE) AS DROP_DATE,
LEFT(CAMP.CAMPAIGN_NUMBER, 4) AS CAMP_NBR, CAMP.CAMPAIGN_NUMBER,

ISNULL(RO.AGE, DTL.IB_AGE) AS IB_AGE, RO.AGE, 
ISNULL(RO.MARRIED, DTL.MARITALSTATUS) AS MARITALSTATUS, 
ISNULL(RO.MEM_NBR, DTL.MEM_NBR) AS MEM_NBR,
ISNULL(RO.GS_LAST_ACTIVITY_DATE, DTL.GS_LAST_ACTIVITY_DATE) AS GS_LAST_ACTIVITY_DATE, 
ISNULL(RO.GS_EXPIRE_DATE, RO.GS_EXPIRE_DATE) AS GS_EXPIRE_DATE, 
ISNULL(RO.GS_MEMBERSHIP_STATUS, DTL.GS_MEMBERSHIP_STATUS) AS GS_MEMBERSHIP_STATUS, 
ISNULL(RO.GS_MEMBER_TYPE, DTL.GS_MEMBER_TYPE) AS GS_MEMBER_TYPE,
ISNULL(RO.GS_ORIG_START_DATE, DTL.GS_ORIG_START_DATE) AS GS_ORIG_START_DATE, 
ISNULL(RO.GS_CURRENT_START_DATE, DTL.GS_CURRENT_START_DATE) AS GS_CURRENT_START_DATE,
ISNULL(RO.GS_PRIOR_TERM, DTL.GS_PRIOR_TERM) AS GS_PRIOR_TERM, 
ISNULL(RO.GS_CURRENT_TERM, DTL.GS_CURRENT_START_DATE) AS GS_CURRENT_TERM, 
ISNULL(RO.GS_CLUB_TIMES_RENEWED, DTL.GS_CLUB_TIMES_RENEWED) AS GS_CLUB_TIMES_RENEWED, 
ISNULL(RO.GS_ERS_MEMBERSHIP_STATUS, DTL.GS_ERS_MEMBERSHIP_STATUS) AS GS_ERS_MEMBERSHIP_STATUS,
ISNULL(RO.GS_ERS_MEMBER_TYPE, DTL.GS_ERS_MEMBER_TYPE) AS GS_ERS_MEMBER_TYPE, 
ISNULL(RO.GS_ERS_ORIG_START_DATE, DTL.GS_ERS_ORIG_START_DATE) AS GS_ERS_ORIG_START_DATE,
ISNULL(RO.GS_ERS_CURRENT_START_DATE, DTL.GS_ERS_CURRENT_START_DATE) AS GS_ERS_CURRENT_START_DATE, 
ISNULL(RO.GS_ERS_CURRENT_TERM, DTL.GS_ERS_CURRENT_TERM) AS GS_ERS_CURRENT_TERM,
ISNULL(RO.GS_ERS_PRIOR_TERM, DTL.GS_ERS_PRIOR_TERM) AS GS_ERS_PRIOR_TERM, 
ISNULL(RO.GS_ERS_TIMES_RENEWED, DTL.GS_ERS_TIMES_RENEWED) AS GS_ERS_TIMES_RENEWED,
ISNULL(RO.GS_ERS_LAST_ACTIVITY_DT, DTL.GS_ERS_LAST_ACTIVITY_DT) AS GS_ERS_LAST_ACTIVITY_DT, 
ISNULL(RO.CSP_MEMBERSHIP_STATUS, DTL.CSP_MEMBERSHIP_STATUS) AS CSP_MEMBERSHIP_STATUS,
ISNULL(RO.CSP_MEMBER_TYPE, DTL.CSP_MEMBER_TYPE) AS CSP_MEMBER_TYPE, 
ISNULL(RO.CSP_CURRENT_START_DATE, DTL.CSP_CURRENT_START_DATE) AS CSP_CURRENT_START_DATE,
ISNULL(RO.CSP_CURRENT_TERM, DTL.CSP_CURRENT_TERM) AS CSP_CURRENT_TERM, 
ISNULL(RO.CSP_PRIOR_TERM, DTL.CSP_PRIOR_TERM) AS CSP_PRIOR_TERM,
ISNULL(RO.CSP_TIMES_RENEWED, DTL.CSP_TIMES_RENEWED) AS CSP_TIMES_RENEWED, 
ISNULL(RO.CSP_LAST_ACTIVITY_DATE, DTL.CSP_LAST_ACTIVITY_DATE) AS CSP_LAST_ACTIVITY_DATE,
ISNULL(RO.RV_ENTHUSIAST_FLAG, DTL.RV_ENTHUSIAST_FLAG) AS RV_ENTHUSIAST_FLAG,
ISNULL(RO.FISHING_MARINE_FLAG, DTL.FISHING_MARINE_FLAG) AS FISHING_MARINE_FLAG,
ISNULL(RO.HUNT_SHOOT_FLAG, DTL.HUNT_SHOOT_FLAG) AS HUNT_SHOOT_FLAG,
ISNULL(RO.OUTDOOR_FLAG, DTL.OUTDOOR_FLAG) AS OUTDOOR_FLAG,
ISNULL(RO.CW_WARRANTY_PRODUCT, DTL.CW_WARRANTY_PRODUCT) AS CW_WARRANTY_PRODUCT,
ISNULL(RO.WOODALL_SUBS_FLAG, DTL.WOODALL_SUBS_FLAG) AS WOODALL_SUBS_FLAG,
ISNULL(RO.TOTAL_CLAIMS, DTL.TOTAL_CLAIMS) AS TOTAL_CLAIMS,
ISNULL(RO.CREDIT_CARD_HOLDER_DIVISION, DTL.CREDIT_CARD_HOLDER_DIVISION) AS CREDIT_CARD_HOLDER_DIVISION,
ISNULL(RO.ALL_CWH_0_12_AVG_SPEND, DTL.ALL_CWH_0_12_AVG_SPEND) AS ALL_CWH_0_12_AVG_SPEND,
ISNULL(RO.ALL_CWH_0_12_AMT_PURCH_TOTAL, DTL.ALL_CWH_0_12_AMT_PURCH_TOTAL) AS ALL_CWH_0_12_AMT_PURCH_TOTAL,
ISNULL(RO.CW_0_12_AVG_SPEND, DTL.CW_0_12_AVG_SPEND) AS CW_0_12_AVG_SPEND,
ISNULL(RO.CW_TRANS_12_MONTHS, DTL.CW_TRANS_12_MONTHS) AS CW_TRANS_12_MONTHS,
ISNULL(RO.CW_0_12_AMT_PURCH_TOTAL, DTL.CW_0_12_AMT_PURCH_TOTAL) AS CW_0_12_AMT_PURCH_TOTAL,
ISNULL(RO.GANDER_0_12_AVG_SPEND, DTL.GANDER_0_12_AVG_SPEND) AS GANDER_0_12_AVG_SPEND,
ISNULL(RO.GANDER_TRANS_12_MONTHS, DTL.GANDER_TRANS_12_MONTHS) AS GANDER_TRANS_12_MONTHS,
ISNULL(RO.GANDER_0_12_AMT_PURCH_TOTAL, DTL.GANDER_0_12_AMT_PURCH_TOTAL) AS GANDER_0_12_AMT_PURCH_TOTAL,
ISNULL(RO.OVERTONS_TRANS_12_MONTHS, DTL.OVERTONS_TRANS_12_MONTHS) AS OVERTONS_TRANS_12_MONTHS,
ISNULL(RO.OVERTONS_0_12_AMT_PURCH_TOTAL, DTL.OVERTONS_0_12_AMT_PURCH_TOTAL) AS OVERTONS_0_12_AMT_PURCH_TOTAL,
ISNULL(RO.ALL_CWH_0_12_PURE_RTL_BUYERS, DTL.ALL_CWH_0_12_PURE_RTL_BUYERS) AS ALL_CWH_0_12_PURE_RTL_BUYERS,
ISNULL(RO.ALL_CWH_13_24_PURE_RTL_BUYERS, DTL.ALL_CWH_13_24_PURE_RTL_BUYERS) AS ALL_CWH_13_24_PURE_RTL_BUYERS,
ISNULL(RO.ALL_CWH_0_12_INT_MO_BUYERS, DTL.ALL_CWH_0_12_INT_MO_BUYERS) AS ALL_CWH_0_12_INT_MO_BUYERS,
ISNULL(RO.ALL_CWH_13_24_INT_MO_BUYERS, DTL.ALL_CWH_13_24_INT_MO_BUYERS) AS ALL_CWH_13_24_INT_MO_BUYERS,
ISNULL(RO.ALL_CWH_0_12_COMBO_BUYERS, DTL.ALL_CWH_0_12_COMBO_BUYERS) AS ALL_CWH_0_12_COMBO_BUYERS,
ISNULL(RO.ALL_CWH_13_24_COMBO_BUYERS, DTL.ALL_CWH_13_24_COMBO_BUYERS) AS ALL_CWH_13_24_COMBO_BUYERS,
ISNULL(RO.CATALOG_REQ_0_24M, DTL.CATALOG_REQ_0_24M) AS CATALOG_REQ_0_24M,
ISNULL(RO.EVENT_ACTIVITY_0_12M, DTL.EVENT_ACTIVITY_0_12M) AS EVENT_ACTIVITY_0_12M,
ISNULL(RO.RV_SALES_0_12M, DTL.RV_SALES_0_12M) AS RV_SALES_0_12M,
ISNULL(RO.RV_SALES_13_24M, DTL.RV_SALES_13_24M) AS RV_SALES_13_24M,
ISNULL(RO.RV_SERVICE_0_12M, DTL.RV_SERVICE_0_12M) AS RV_SERVICE_0_12M,
ISNULL(RO.RV_SERVICE_13_24M, DTL.RV_SERVICE_13_24M) AS RV_SERVICE_13_24M,
ISNULL(RO.GSC_ACTIVE, DTL.GSC_ACTIVE) AS GSC_ACTIVE,
ISNULL(RO.GSC_24M_EXPIRES, DTL.GSC_24M_EXPIRES) AS GSC_24M_EXPIRES,
ISNULL(RO.CTC_ACTIVE, DTL.CTC_ACTIVE) AS CTC_ACTIVE,
ISNULL(RO.CTC_24M_EXPIRES, DTL.CTC_24M_EXPIRES) AS CTC_24M_EXPIRES,
ISNULL(RO.RA_DLR_ACTIVE, DTL.RA_DLR_ACTIVE) AS RA_DLR_ACTIVE,
ISNULL(RO.RA_DLR_24M_EXPIRES, DTL.RA_DLR_24M_EXPIRES) AS RA_DLR_24M_EXPIRES,
ISNULL(RO.GSRA_AUTO_ACTIVE, DTL.GSRA_AUTO_ACTIVE) AS GSRA_AUTO_ACTIVE,
ISNULL(RO.GSRA_RV_ACTIVE, DTL.GSRA_RV_ACTIVE) AS GSRA_RV_ACTIVE,
ISNULL(RO.GSRA_RV_24M_EXPIRES, DTL.GSRA_RV_24M_EXPIRES) AS GSRA_RV_24M_EXPIRES,
ISNULL(RO.RA_MANUF_ACTIVE, DTL.RA_MANUF_ACTIVE) AS RA_MANUF_ACTIVE,
ISNULL(RO.RA_MANUF_24M_EXPIRES, DTL.RA_MANUF_24M_EXPIRES) AS RA_MANUF_24M_EXPIRES,
ISNULL(RO.RA_PARTNER_ACTIVE, DTL.RA_PARTNER_ACTIVE) AS RA_PARTNER_ACTIVE,
ISNULL(RO.ESP_ACTIVE, DTL.ESP_ACTIVE) AS ESP_ACTIVE,
ISNULL(RO.ESP_24M_LAPSED, DTL.ESP_24M_LAPSED) AS ESP_24M_LAPSED,
ISNULL(RO.ASU_ESP_ACTIVE_POLICIES, DTL.ASU_ESP_ACTIVE_POLICIES) AS ASU_ESP_ACTIVE_POLICIES,
ISNULL(RO.ASU_ESP_24M_LAPSED_POLICIES, DTL.ASU_ESP_24M_LAPSED_POLICIES) AS ASU_ESP_24M_LAPSED_POLICIES,
ISNULL(RO.GSTA_ACTIVE, DTL.GSTA_ACTIVE) AS GSTA_ACTIVE,
ISNULL(RO.GSTA_24M_EXPIRES, DTL.GSTA_24M_EXPIRES) AS GSTA_24M_EXPIRES,
ISNULL(RO.GSTA_DLR_ACTIVE, DTL.GSTA_DLR_ACTIVE) AS GSTA_DLR_ACTIVE,
ISNULL(RO.GSTA_DLR_24M_EXPIRES, DTL.GSTA_DLR_24M_EXPIRES) AS GSTA_DLR_24M_EXPIRES,
ISNULL(RO.GSRA_AUTO_24M_EXPIRES, DTL.GSRA_AUTO_24M_EXPIRES) AS GSRA_AUTO_24M_EXPIRES,
ISNULL(RO.AGIA_24M_LAPSED_POLICIES, DTL.AGIA_24M_LAPSED_POLICIES) AS AGIA_24M_LAPSED_POLICIES,
ISNULL(RO.RV_MAG_ACTIVE, DTL.RV_MAG_ACTIVE) AS RV_MAG_ACTIVE,
ISNULL(RO.RV_MAG_24M_EXPIRES, DTL.RV_MAG_24M_EXPIRES) AS RV_MAG_24M_EXPIRES,
ISNULL(RO.CC_ALL_FLAVORS_OPEN_ACCTS, DTL.CC_ALL_FLAVORS_OPEN_ACCTS) AS CC_ALL_FLAVORS_OPEN_ACCTS,
ISNULL(RO.PPP_ACTIVE_POLICIES, DTL.PPP_ACTIVE_POLICIES) AS PPP_ACTIVE_POLICIES,
ISNULL(RO.PPP_24M_LAPSED_POLICIES, DTL.PPP_24M_LAPSED_POLICIES) AS PPP_24M_LAPSED_POLICIES,
ISNULL(RO.Address_Line_1, DTL.ADDRESS_LINE1) AS ADDRESS_LINE1,
ISNULL(RO.ADDRESS_LINE_2, DTL.ADDRESS_LINE2) AS ADDRESS_LINE2,
ISNULL(RO.ZIP_5, DTL.POSTAL_CODE_5) AS POSTAL_CODE_5,
ISNULL(RO.RV_TYPE, DTL.RV_TYPE) AS RV_TYPE,

ZIP.MED_HOME_VALUE, ZIP.MEDIAN_INCOME, ZIP.DENSITY, ZIP.LAT, ZIP.LNG, ZIP.RISK_RATNG,

CASE 
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Very Low' THEN 0 WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively Low' THEN 1 
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively Moderate' THEN 2 WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively High' THEN 3
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Very High' THEN 4 WHEN ISNULL(ZIP.RISK_RATNG, '') = '' THEN 5 ELSE 5 END AS NRI_RR,

REG.DIV_NBR, SI.STATE_INDEX_NBR, RUCA.RUCA1, T.MEAN_ST_TEMP

FROM D2CDIRECTMAIL.DBO.GOODSAMDM_CAMPAIGNHISTORY_NEW CAMP WITH(NOLOCK) 
LEFT JOIN [D2CDIRECTMAIL].[DBO].[GOODSAM_ROLLOUT_CANDIDATE_ALL_DETAILS] DTL WITH(NOLOCK) ON CAMP.OMNIVIEW_IND_ID = DTL.OMNIVIEW_IND_ID AND DTL.OMNIVIEW_IND_ID IS NOT NULL 
LEFT JOIN [D2CDIRECTMAIL].[DBO].[GS_EVENT_FOR_RO] RO WITH(NOLOCK) ON CAMP.OMNIVIEW_IND_ID = RO.IID AND LEFT(CAMP.CAMPAIGN_NUMBER, 4) = LEFT(RO.CAMPAIGN_NUMBER, 4)

LEFT JOIN [D2CDIRECTMAIL].DBO.GOODSAM_ALLDM_QUOTES Q WITH(NOLOCK) ON CAMP.OMNIVIEW_IND_ID = Q.MAILEDIID 
AND LEFT(CAMP.CAMPAIGN_NUMBER, 4) = LEFT(Q.CAMPAIGN_NUMBER, 4)

LEFT JOIN (SELECT Q.CAMP_NBR, Q.OMNIVIEW_IND_ID, Q.QUOTEMADE, Q.TM,
CASE WHEN (SUM(CASE WHEN Q.QUOTEMADE >= 1 THEN 1 ELSE 0 END) OVER (PARTITION BY Q.OMNIVIEW_IND_ID ORDER BY Q.OMNIVIEW_IND_ID, Q.CAMP_NBR) - 1) < 0 THEN 0
ELSE (SUM(CASE WHEN Q.QUOTEMADE >= 1 THEN 1 ELSE 0 END) OVER (PARTITION BY Q.OMNIVIEW_IND_ID ORDER BY Q.OMNIVIEW_IND_ID, Q.CAMP_NBR) - 1) END AS TQ

FROM (
SELECT DISTINCT LEFT(CAMP.CAMPAIGN_NUMBER, 4) AS CAMP_NBR, CAMP.OMNIVIEW_IND_ID, SUM(ISNULL(Q.QUOTEMADE, 0)) AS QUOTEMADE,
COUNT(CAMP.OMNIVIEW_IND_ID) OVER (PARTITION BY CAMP.OMNIVIEW_IND_ID ORDER BY CAMP.OMNIVIEW_IND_ID, LEFT(CAMP.CAMPAIGN_NUMBER, 4)) - 1 AS TM
FROM D2CDIRECTMAIL.DBO.GOODSAMDM_CAMPAIGNHISTORY_NEW CAMP WITH(NOLOCK) 
LEFT JOIN [D2CDIRECTMAIL].DBO.GOODSAM_ALLDM_QUOTES Q WITH(NOLOCK) ON CAMP.OMNIVIEW_IND_ID = Q.MAILEDIID
AND LEFT(CAMP.CAMPAIGN_NUMBER, 4) = LEFT(Q.CAMPAIGN_NUMBER, 4)
WHERE LEFT(CAMP.CAMPAIGN_NUMBER, 4) > 2300
GROUP BY LEFT(CAMP.CAMPAIGN_NUMBER, 4), CAMP.OMNIVIEW_IND_ID
) Q
) TM ON LEFT(CAMP.CAMPAIGN_NUMBER, 4) = TM.CAMP_NBR AND CAMP.OMNIVIEW_IND_ID = TM.OMNIVIEW_IND_ID

LEFT JOIN (SELECT * FROM (SELECT O.OMNIINDIVIDUALIDENTIFIER, O.AGE, O.ROWUPDATEDATETIME,
ROW_NUMBER() OVER (PARTITION BY O.OMNIINDIVIDUALIDENTIFIER ORDER BY O.ROWUPDATEDATETIME DESC) AS R
FROM D2CDIRECTMAIL.STAGE.OMNI_IDS O WITH(NOLOCK) 
) O WHERE 1=1 AND O.R=1 AND O.OMNIINDIVIDUALIDENTIFIER IS NOT NULL AND O.OMNIINDIVIDUALIDENTIFIER <> '' AND O.AGE IS NOT NULL AND O.AGE <> ''
) O ON CAMP.OMNIVIEW_IND_ID = O.OMNIINDIVIDUALIDENTIFIER

LEFT JOIN [D2CDirectMail].[Stage].[STATE_INDEX] SI WITH(NOLOCK) ON SI.GOVSTATE = CAMP.[STATE]
LEFT JOIN D2CDIRECTMAIL.STAGE.US_REGIONS REG WITH(NOLOCK) ON CAMP.[STATE] = REG.ST
LEFT JOIN D2CDIRECTMAIL.STAGE.RUCA_ZIPCODE_CD RUCA WITH(NOLOCK) ON CAMP.ZIP_5 = RUCA.ZIP_CODE

LEFT JOIN (SELECT * FROM (
SELECT ZIP.ZIPCODE, ZIP.MED_HOME_VALUE, ZIP.MEDIAN_INCOME, ZIP.DENSITY, ZIP.LAT, ZIP.LNG, ZIP.RISK_RATNG,
CASE 
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Very Low' THEN 0 WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively Low' THEN 1 
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively Moderate' THEN 2 WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Relatively High' THEN 3
WHEN ISNULL(ZIP.RISK_RATNG, '') = 'Very High' THEN 4 WHEN ISNULL(ZIP.RISK_RATNG, '') = '' THEN 5 ELSE 5 END AS NRI_RR,
ROW_NUMBER() OVER (PARTITION BY ZIP.ZIPCODE, ZIP.STATE ORDER BY ZIP.STATE DESC) AS R
FROM D2CDIRECTMAIL.DBO.COMBINED_ZIP_DATA ZIP WITH(NOLOCK)
) ZIP WHERE ZIP.R=1) ZIP ON CAMP.ZIP_5 = ZIP.ZIPCODE

LEFT JOIN (
SELECT DISTINCT T.ST, T.[MONTH], ROUND(SUM(CAST(T.MONTHLY_MEAN_FROM_1901_TO_2000 AS FLOAT)) / COUNT(DISTINCT  T.[YEAR] + T.[MONTH]), 2) AS MEAN_ST_TEMP
FROM [D2CDIRECTMAIL].[DBO].[AVE_MONTH_TEMP_ST_1950-2022] T WITH(NOLOCK)
GROUP BY T.ST, T.[MONTH]
) T ON CAMP.[STATE] = T.ST AND SUBSTRING(CAST(CAMP.DROP_DATE AS VARCHAR), 6, 2) = T.[month]

WHERE 1=1
AND CAMP.HOLD = 0
AND (UPPER(CAMP.MAILTYPE) IN ('MAILED-SELECTED') 
OR (UPPER(CAMP.MAILTYPE) IN ('MAILED-RANDOM') AND Q.QUOTEMADE = 1)
)

AND CAMP.OMNIVIEW_IND_ID IS NOT NULL
AND UPPER(CAMP.PROGRAM) IN ('ROLLOUT')
) CUST WHERE CUST.R=1
) CUST 

ORDER BY CUST.CAMPAIGN_NUMBER DESC
"""

# ==========================================================
# ✅ 4️⃣ Connect to SQL Server
# ==========================================================
server = "PWVDDBSQLA055.NGIC.COM"
conn_str = (
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={server};"
    f"Trusted_Connection=yes;"
)

print("🔗 Connecting to database...")
try:
    conn = pyodbc.connect(conn_str)
    print("✅ SQL connection established.")
except Exception as e:
    raise SystemExit(f"❌ Database connection failed: {e}")

# ==========================================================
# ✅ 5️⃣ Data Extraction
# ==========================================================
print("📥 Running query and loading dataset...")
try:
    data = pd.read_sql(qry, conn)
    print(f"✅ Data loaded successfully — {len(data)} rows × {len(data.columns)} columns")
except Exception as e:
    raise SystemExit(f"❌ SQL query failed: {e}")

# Close connection
conn.close()

# ==========================================================
# ✅ 6️⃣ Optional Parallel Processing Example
# ==========================================================
def summarize_chunk(chunk):
    """Example function to process each chunk of the dataset."""
    return {
        "rows": len(chunk),
        "mean_home_value": chunk["MED_HOME_VALUE"].mean(),
        "avg_temp": chunk["MEAN_ST_TEMP"].mean(),
    }

chunk_size = max(1, len(data) // num_cores)
chunks = [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

print("⚙️ Running parallel summarization...")
with ThreadPoolExecutor(max_workers=num_cores) as executor:
    summaries = list(executor.map(summarize_chunk, chunks))

summary_df = pd.DataFrame(summaries)
print("\n📊 Summary of chunks:")
print(summary_df)

⚠️ SSL verification is disabled — use only in a secure internal environment.
💻 Using 1 cores for data processing
🔗 Connecting to database...
✅ SQL connection established.
📥 Running query and loading dataset...


C:\Users\i231188\AppData\Local\Temp\ipykernel_17540\3192233498.py:458: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(qry, conn)


✅ Data loaded successfully — 1141244 rows × 119 columns
⚙️ Running parallel summarization...

📊 Summary of chunks:
      rows  mean_home_value   avg_temp
0  1141244    164344.731058  56.958042


In [6]:
# ==========================================================
# ✅ Derive Monthly QUOTE Performance (Total + Direct + Mailed)
# ==========================================================
import pandas as pd

# --- Normalize MAIL_YM to YYYY-MM ---
data["MAIL_YM"] = pd.to_datetime(data["MAIL_YM"], errors="coerce").dt.strftime("%Y-%m")

# --- Compute QUOTE flag ---
if "DIR_QT_FLG" in data.columns:
    data["QUOTE"] = ((data["QUOTE"] == 1) & (data["DIR_QT_FLG"] == 1)).astype(int)
elif "DIR_IND" in data.columns:
    data["QUOTE"] = ((data["QUOTE"] == 1) & (data["DIR_IND"] == 1)).astype(int)
else:
    raise KeyError("Neither DIR_QT_FLG nor DIR_IND found in dataset.")

# --- Aggregate by MAIL_YM ---
quote_trends_df = (
    data.groupby("MAIL_YM", dropna=True)
    .agg(
        MAILED_COUNT=("IID_TXT", "count"),        # total mailed records
        IID_COUNT=("IID_TXT", "nunique"),         # unique customers mailed
        QUOTE_COUNT=("QUOTE", "sum"),             # total quotes
        DIRECT_QUOTE=("DIR_QT_FLG", lambda x: ((data.loc[x.index, "QUOTE"] == 1) & (x == 1)).sum())
    )
    .reset_index()
)

# --- Compute Rates ---
quote_trends_df["QUOTE_RATE"] = (
    quote_trends_df["QUOTE_COUNT"] / quote_trends_df["MAILED_COUNT"]
).fillna(0).round(4)

quote_trends_df["DIRECT_RATE"] = (
    quote_trends_df["DIRECT_QUOTE"] / quote_trends_df["MAILED_COUNT"]
).fillna(0).round(4)

# --- Sort descending ---
quote_trends_df = quote_trends_df.sort_values("MAIL_YM", ascending=False)

# --- Print results ---
print("\n📈 QUOTE Trends (Total + Direct + Mailed):")
print(quote_trends_df[["MAIL_YM", "MAILED_COUNT", "QUOTE_COUNT", "DIRECT_QUOTE", "QUOTE_RATE", "DIRECT_RATE"]].head(10))


📈 QUOTE Trends (Total + Direct + Mailed):
    MAIL_YM  MAILED_COUNT  QUOTE_COUNT  DIRECT_QUOTE  QUOTE_RATE  DIRECT_RATE
22  2025-10         40042          113           113      0.0028       0.0028
21  2025-08         40089          135           135      0.0034       0.0034
20  2025-06         40129          150           150      0.0037       0.0037
19  2025-04         40108          166           166      0.0041       0.0041
18  2025-02         40082          188           188      0.0047       0.0047
17  2024-12         39808          178           178      0.0045       0.0045
16  2024-10         39628          198           198      0.0050       0.0050
15  2024-08         39849          202           202      0.0051       0.0051
14  2024-06         39699          196           196      0.0049       0.0049
13  2024-04         39717          112           112      0.0028       0.0028


In [7]:
# ==========================================================
# ✅ 10️⃣ HITL Analysis with Prompt Chaining (Total + Direct + Mailed QUOTES)
# ==========================================================
import pandas as pd
import time
from openai import OpenAI

# --- 0️⃣ Setup API key (hardcoded for local use only) ---
API_KEY = "*"
client = OpenAI(api_key=API_KEY)

# ==========================================================
# ✅ 1️⃣ Derive QUOTE, DIRECT_QUOTE, and MAILED_COUNT Trends
# ==========================================================
if all(col in data.columns for col in ["QUOTE", "DIR_QT_FLG", "MAIL_YM", "IID_TXT"]):
    data["MAIL_YM"] = pd.to_datetime(data["MAIL_YM"], errors="coerce").dt.strftime("%Y-%m")
    data["QUOTE"] = ((data["QUOTE"] == 1) & (data["DIR_QT_FLG"] == 1)).astype(int)

    quote_trends_df = (
        data.groupby("MAIL_YM", dropna=True)
        .agg(
            MAILED_COUNT=("IID_TXT", "count"),
            IID_COUNT=("IID_TXT", "nunique"),
            QUOTE_COUNT=("QUOTE", "sum"),
            DIRECT_QUOTE=("DIR_QT_FLG", lambda x: ((data.loc[x.index, "QUOTE"] == 1) & (x == 1)).sum())
        )
        .reset_index()
    )

    quote_trends_df["QUOTE_RATE"] = (
        quote_trends_df["QUOTE_COUNT"] / quote_trends_df["MAILED_COUNT"]
    ).fillna(0).round(4)

    quote_trends_df["DIRECT_RATE"] = (
        quote_trends_df["DIRECT_QUOTE"] / quote_trends_df["MAILED_COUNT"]
    ).fillna(0).round(4)

    quote_trends_df = quote_trends_df.sort_values("MAIL_YM", ascending=False)
    quote_trends_recent = quote_trends_df.head(3)
    mailed_trends = dict(zip(quote_trends_recent["MAIL_YM"], quote_trends_recent["MAILED_COUNT"].astype(int)))
    quote_trends = dict(zip(quote_trends_recent["MAIL_YM"], quote_trends_recent["QUOTE_RATE"].astype(float)))
    direct_quote_trends = dict(zip(quote_trends_recent["MAIL_YM"], quote_trends_recent["DIRECT_RATE"].astype(float)))
else:
    mailed_trends = {"2025-09": 40000, "2025-10": 39500, "2025-11": 38500}
    quote_trends = {"2025-09": 0.082, "2025-10": 0.075, "2025-11": 0.071}
    direct_quote_trends = {"2025-09": 0.041, "2025-10": 0.037, "2025-11": 0.034}

print("\n📈 QUOTE Trends (Mailed + Total + Direct):")
print(quote_trends_df[["MAIL_YM", "MAILED_COUNT", "QUOTE_COUNT", "DIRECT_QUOTE", "QUOTE_RATE", "DIRECT_RATE"]].head(6))
print("\n📊 Recent MAILED Counts:", mailed_trends)
print("📊 Recent QUOTE Rates:", quote_trends)
print("📊 Recent DIRECT Rates:", direct_quote_trends)

# ==========================================================
# ✅ 1.5️⃣ Feature Gap Audit (detect unused but valuable attributes)
# ==========================================================
features_used = [
    "TQ", "GS_PRIOR_TERM", "RV_SERVICE_0_12M", "ESP_ACTIVE", "MARRIED_HOMEOWNER",
    "RA_DLR_24M_EXPIRES", "CombinedFlags", "PPP_ACTIVE_POLICIES", "GSRA_RV_24M_EXPIRES",
    "GSTA_DLR_ACTIVE", "ASU_ESP_ACTIVE_POLICIES", "GSTA_ACTIVE", "PPP_24M_LAPSED_POLICIES",
    "RV_TYPE_NBR", "CW_WARRANTY_PRODUCT", "DIV_NBR", "STATE_INDEX_NBR", "RUCA",
    "MEAN_ST_TEMP", "ZIP_3", "DENSITY", "MEDIAN_INCOME"
]

if "data" in locals():
    all_features = list(data.columns)
    unused_features = [f for f in all_features if f not in features_used]
    excluded_patterns = ["IID", "QUOTE", "BOUND", "MAIL", "KEYCODE", "CAMP", "DATE", "WK", "TARGET"]
    candidate_features = [
        f for f in unused_features
        if not any(pat in f.upper() for pat in excluded_patterns)
    ]

    print(f"\n🔍 Feature Gap Audit — {len(candidate_features)} unused attributes detected.")
    print(candidate_features[:25])

    prompt_gap = f"""
    The current model uses: {', '.join(features_used)}.
    Unused but available fields include: {', '.join(candidate_features[:40])}.
    Which of these unused fields could be most predictive of QUOTE or DIRECT_QUOTE outcomes?
    Focus on recency (e.g., GS_LAST_ACTIVITY_DATE_DIFF), spending, renewal, or engagement indicators.
    Provide a short ranked explanation of top potential features.
    """

    try:
        resp_gap = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert feature engineer in insurance marketing analytics."},
                {"role": "user", "content": prompt_gap}
            ],
            temperature=0.3
        )
        feature_gap_summary = resp_gap.choices[0].message.content
        print("\n📊 Step 1.5: Feature Gap Summary:\n")
        print(feature_gap_summary)
    except Exception as e:
        feature_gap_summary = f"⚠️ Feature gap reasoning failed: {e}"
        print(feature_gap_summary)
else:
    feature_gap_summary = "⚠️ Feature audit skipped — data not loaded."

# ==========================================================
# ✅ 2️⃣ Last Model Performance (Campaign 2543)
# ==========================================================
last_model_results = {
    "campaign": 2543,
    "recall": 0.7593,
    "specificity": 0.5174,
    "precision": 0.0048,
    "f1_score": 0.0095,
    "auc": 0.6748,
    "confusion_matrix": [[27366, 25528], [39, 123]]
}

model_metrics = {
    "train_accuracy": 0.9212,
    "train_recall": 0.9896,
    "test_accuracy": 0.5181,
    "test_recall": last_model_results["recall"],
    "test_auc": last_model_results["auc"]
}

# ==========================================================
# ✅ 3️⃣ Step 1: Model Audit (Mailed + Direct Quote Context)
# ==========================================================
prompt_1 = f"""
You are a data scientist reviewing a marketing quote prediction model for mailed campaigns.

Target: QUOTE = 1
Features used: {', '.join(features_used)}
Unused features analysis: {feature_gap_summary}

Recent MAILED volumes: {mailed_trends}
Recent overall QUOTE rates: {quote_trends}
Recent DIRECT_QUOTE rates: {direct_quote_trends}

Train Accuracy={model_metrics['train_accuracy']}, Recall={model_metrics['train_recall']}
Test Accuracy={model_metrics['test_accuracy']}, Recall={model_metrics['test_recall']}, AUC={model_metrics['test_auc']}

Confusion Matrix:
TN={last_model_results['confusion_matrix'][0][0]},
FP={last_model_results['confusion_matrix'][0][1]},
FN={last_model_results['confusion_matrix'][1][0]},
TP={last_model_results['confusion_matrix'][1][1]}

Please analyze:
- How does mailed volume affect QUOTE and DIRECT_QUOTE conversion performance?
- Which unused or underused attributes from the feature gap audit might strengthen the model?
- What potential weaknesses or data drift might explain performance differences?
- Recommend actionable improvements to boost DIRECT_QUOTE conversion and campaign efficiency.
"""

resp1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an experienced data science auditor focused on marketing response modeling."},
        {"role": "user", "content": prompt_1}
    ],
    temperature=0.25
)
summary_1 = resp1.choices[0].message.content
print("\n📊 Step 1: Model Audit Summary:\n")
print(summary_1)

# ==========================================================
# ✅ 4️⃣ Step 2: Deep-Dive – Improving DIRECT_QUOTES
# ==========================================================
prompt_2 = f"""
Based on the following model audit and feature gap insights:

{summary_1}

Now act as a lead ML engineer and provide:
1. New feature engineering ideas (recency, renewal, or behavioral).
2. Recommended experiments (segmentation, calibration, sampling strategies).
3. Hypotheses about low DIRECT_QUOTE performance.
4. A prioritized roadmap balancing recall, precision, and campaign ROI.
"""

resp2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a senior ML lead mentoring a marketing data science team."},
        {"role": "user", "content": prompt_2}
    ],
    temperature=0.35
)

summary_2 = resp2.choices[0].message.content
print("\n🧠 Step 2: Deep-Dive Recommendations (DIRECT QUOTES Focus):\n")
print(summary_2)

# ==========================================================
# ✅ 5️⃣ Step 3: Wrap-Up
# ==========================================================
combined_output = f"=== MODEL AUDIT ===\n{summary_1}\n\n=== DIRECT QUOTE ACTION PLAN ===\n{summary_2}"
print("\n✅ Chained HITL Analysis Complete!\n")


📈 QUOTE Trends (Mailed + Total + Direct):
    MAIL_YM  MAILED_COUNT  QUOTE_COUNT  DIRECT_QUOTE  QUOTE_RATE  DIRECT_RATE
22  2025-10         40042          113           113      0.0028       0.0028
21  2025-08         40089          135           135      0.0034       0.0034
20  2025-06         40129          150           150      0.0037       0.0037
19  2025-04         40108          166           166      0.0041       0.0041
18  2025-02         40082          188           188      0.0047       0.0047
17  2024-12         39808          178           178      0.0045       0.0045

📊 Recent MAILED Counts: {'2025-10': 40042, '2025-08': 40089, '2025-06': 40129}
📊 Recent QUOTE Rates: {'2025-10': 0.0028, '2025-08': 0.0034, '2025-06': 0.0037}
📊 Recent DIRECT Rates: {'2025-10': 0.0028, '2025-08': 0.0034, '2025-06': 0.0037}

🔍 Feature Gap Audit — 76 unused attributes detected.
['DIR_OR_IND_QT', 'DIR_QT_FLG', 'HOMEOWNER', 'RA_ACTIVE_TQ', 'AGE', 'ALL_CWH_0_12_AVG_SPEND', 'POSTAL_CODE_5', 'ZIP_